In [ ]:
import numpy as np
import pandas as pd


# Betting Both Sides with Boost

Profit Must be greater than 0 on both sides  
$B_1 + B_1O_1 -B_1- B_2 >= 0$  
$B_2 +B_2O_2-B_1-B_2 >=0$

$B_1O_1 - B_2 = B_2O_2 -B_1$


$\large
B_2 = \frac{B_1(O_1 + 1)}{O_2 + 1}
$

Subject to $B_1 + B_1O_1 - B_2 -B_1 >= 0$  

Sub in $B_2$

$\large
B_1O_1 >= \frac{B_1(O_1 + 1)}{O_2 + 1
}$

$\large
\frac{O_1}{O_1 + 1} >= \frac{1}{O_2 + 1}
$

What boost $x$ to $O_1$ will satistfy this inequality?
$\large
\frac{O_1x}{O_1x + 1} >= \frac{1}{O_2 + 1}
$

$\large
x = \frac{1}{O_1O_2}
$


# Betting Both Sides With a Free Bet
Profit Must be greater than 0 on both sides  
$B_fO_1 - B_2 >= 0$  
$B_2 +B_2O_2-B_2 >=0$

$\large
B_2 = \frac{B_fO_1}{O_2 + 1}
$

$\large
Profit_{pct} = \frac{Profit}{Cost}
$
$\large
Profit_{pct} = (B_fO_1 - B_2) \frac{1}{B_f}
$
$\large
Profit_{pct} = O_1 -  \frac{O_1}{O_2 + 1}
$

In [ ]:
op = np.arange(100,1000,10)
on = -1 * np.arange(100,1000,10)
min_boost =np.outer(op,on) * np.nan
free_wager_to_profit = np.outer(op,on) * np.nan
aa, bb =np.meshgrid(op, on)
o1 = (aa/100)
o2 = -1*(100/bb)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')
sns.set_palette('cividis')
%matplotlib inline

In [ ]:
# Boost PCT
plt.figure(figsize=(16,9))
zz = 1 / (o1 * o2)
plt.contourf(aa,bb, zz,levels=np.arange(0,1.1,0.01), cmap='jet')
plt.colorbar(label='Boost % to Profit')
plt.xlabel('Odds Side 1')
plt.ylabel('Odds Side 2')
plt.grid()

In [ ]:
plt.figure(figsize=(16,9))
free_bet = 300
winnings = free_bet * o1
b2 = (free_bet * o1 / (1 +o2))
profit = winnings - b2
percent_profit = profit / free_bet
plt.contourf(aa, bb, percent_profit,levels=np.arange(0,1.1,0.001), cmap='jet')
plt.colorbar(label='$\\frac{Profit}{Free Bet}$')
plt.xlabel('Odds Side 1')
plt.ylabel('Odds Side 2')
plt.grid()